# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846554684819                   -0.70    4.2         
  2   -7.852283057969       -2.24       -1.53    1.0   23.1ms
  3   -7.852612652512       -3.48       -2.56    1.5   25.7ms
  4   -7.852645950629       -4.48       -2.90    2.5   30.5ms
  5   -7.852646522670       -6.24       -3.21    1.2   24.0ms
  6   -7.852646679896       -6.80       -4.36    1.0   70.3ms
  7   -7.852646686689       -8.17       -4.98    2.5   32.4ms
  8   -7.852646686726      -10.44       -5.50    1.2   24.4ms
  9   -7.852646686729      -11.49       -5.74    1.2   25.0ms
 10   -7.852646686730      -12.06       -6.79    1.0   23.3ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846897007893                   -0.70           4.5         
  2   -7.852523480303       -2.25       -1.64   0.80    2.0    262ms
  3   -7.852634954972       -3.95       -2.73   0.80    1.0   22.3ms
  4   -7.852646440451       -4.94       -3.23   0.80    2.2   29.1ms
  5   -7.852646666285       -6.65       -4.03   0.80    1.0   21.6ms
  6   -7.852646686333       -7.70       -4.71   0.80    1.8   25.7ms
  7   -7.852646686712       -9.42       -5.39   0.80    1.5   24.8ms
  8   -7.852646686729      -10.77       -6.40   0.80    1.2   23.7ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.404689e+01     3.507982e+00
 * time: 0.061927080154418945
     1     1.392624e+00     1.758909e+00
 * time: 0.29290318489074707
     2    -1.390451e+00     1.987786e+00
 * time: 0.31646203994750977
     3    -3.555758e+00     1.879709e+00
 * time: 0.41481804847717285
     4    -4.900150e+00     1.623786e+00
 * time: 0.4485201835632324
     5    -6.589335e+00     9.835665e-01
 * time: 0.4825620651245117
     6    -7.252570e+00     4.661039e-01
 * time: 0.5157961845397949
     7    -7.590098e+00     2.519361e-01
 * time: 0.5396950244903564
     8    -7.739936e+00     1.302259e-01
 * time: 0.5629651546478271
     9    -7.797157e+00     1.076335e-01
 * time: 0.586251974105835
    10    -7.827288e+00     4.359180e-02
 * time: 0.6095561981201172
    11    -7.840700e+00     3.890115e-02
 * time: 0.6330270767211914
    12    -7.844538e+00     4.153870e-02
 * time: 0.6560690402984619
    13    -7.845305e+00     4.634945e-02
 * time: 0.689167

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846806972012                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645892305                   -1.64         
  2   -7.852646686730       -6.10       -3.71    1.94s
  3   -7.852646686730      -13.27       -7.23    189ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.5686321297433786e-7
|ρ_newton - ρ_scfv| = 6.09608828673496e-7
|ρ_newton - ρ_dm|   = 6.333462738272936e-10
